# Lab 5

## Context Engineering

We want to develop several types of Memory:

1. Long Term Memory - graph

A knowledge graph as a persistent store of entities

2. Long Term Memory - knowledge

A RAG database of Q&A and any other useful information

3. Permanent context

Summary and linkedin profile included in everything

4. FAQ

A list of questions and answers

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from pathlib import Path
from datetime import datetime
load_dotenv(override=True)

## Websites where you can find MCP Servers


- https://mcp.so
- https://glama.ai
- https://smithery.ai

## First, look at this Knowledge Graph MCP server built on libsql

https://glama.ai/mcp/servers/@joleyline/mcp-memory-libsql

In [ ]:
file_path = Path("memory") / Path("graph.db")
url = f"file:{file_path.absolute()}"

memory_graph_params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": url}}

In [ ]:
async with MCPServerStdio(params=memory_graph_params, client_session_timeout_seconds=30) as memory_graph:
    memory_graph_tools = await memory_graph.session.list_tools()

memory_graph_tools.tools

## Next, here is a Vector Store RAG memory built on Qdrant

https://glama.ai/mcp/servers/@qdrant/mcp-server-qdrant

In [ ]:
long_term_path = Path("memory") / Path("knowledge")


memory_rag_params = {
    "command": "uvx",
    "args": ["mcp-server-qdrant"],
    "env": {
        "QDRANT_LOCAL_PATH": str(long_term_path.absolute()),
        "COLLECTION_NAME": "knowledge",
        "EMBEDDING_MODEL": "sentence-transformers/all-MiniLM-L6-v2"
    }
}

In [ ]:
async with MCPServerStdio(params=memory_rag_params, client_session_timeout_seconds=30) as memory_rag:
    memory_rag_tools = await memory_rag.session.list_tools()

memory_rag_tools.tools

## And a recent friend

In [ ]:
question_params = {"command": "uv", "args": ["run", "questions_mcp_server.py"]}
async with MCPServerStdio(params=question_params, client_session_timeout_seconds=30) as question_server:
    question_tools = await question_server.session.list_tools()

question_tools.tools

## And now - Context Engineering

In [ ]:
name = "Ed Donner"
first_name = "Ed"
context = f"""

You represent the AI Digital Twin of a human called {name}.

## Your task

You are chatting on {first_name}'s website with a user, who might be a potential customer or a future employer.

You are to engage in conversation with the user, presenting yourself as {name} and answering questions about {first_name} as if you are {first_name}.
If you are pressed, you should be open about actually being a 'digital twin' of {first_name} and your objective is to faithfully represent {first_name}.
You understand that you are in fact an LLM, but your role is to faithfully represent {first_name} and you've been fully briefed and empowered to do so.

## Information about {first_name}

{first_name} is the co-founder and CTO of AI startup Nebula.io.
{first_name} loves coding and experimenting with LLMs.
{first_name} hates cheese.

## {first_name}'s style

{first_name} has an upbeat, friendly and slightly nerdy style. {first_name} tries to engage with users.

## How you should respond

Channel {first_name}'s personality, style and knowledge.
Try to stay focused on professional topics; feel free to engage in other subjects but gently steer the conversation back to professional topics.

## Tools

You have tools to find and store information in Qdrant, which is your long term memory for information.
You have tools to find and store entities and relationships in a graph database; this is your medium term memory.

You should make frequent use of both long and medium term memories.

You also have a tool to load and save answers to questions.
Most importantly, you should always use the tool to record a question that you cannot answer.
This will notify your twin that an answer is needed.

For reference, here is the current date and time:
{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

"""

In [ ]:
print(context)

In [ ]:
with trace("Twin"):
    async with MCPServerStdio(params=memory_rag_params, client_session_timeout_seconds=30) as long_term_memory:
        async with MCPServerStdio(params=memory_graph_params, client_session_timeout_seconds=30) as medium_term_memory:
            async with MCPServerStdio(params=question_params, client_session_timeout_seconds=30) as question_server:
                agent = Agent("Twin", model="gpt-4.1-mini", instructions=context, mcp_servers=[long_term_memory, medium_term_memory, question_server])
                task = [{"role": "user", "content": "Hello, I'm a potential customer. Does Ed like bananas?"}]
                response = await Runner.run(agent, task)
                print(response.final_output)

https://platform.openai.com/traces